# Constraint Optimization


https://developers.google.com/optimization/cp/cp_example

In [1]:
from ortools.sat.python import cp_model

In [2]:
# 1. Declare the model.
model = cp_model.CpModel()

# 2. Create the variables
num_vals = 3
x = model.NewIntVar(0, num_vals - 1, "x")
y = model.NewIntVar(0, num_vals - 1, "y")
z = model.NewIntVar(0, num_vals - 1, "z")

# 3. Create the constraint.
model.Add(x != y)

# 4. Call the solver.
solver = cp_model.CpSolver()
status = solver.Solve(model)
status

4

In [3]:
if status == cp_model.OPTIMAL:
    print("An optimal solution was found.")
elif status == cp_model.FEASIBLE:
    print("A feasible solution ws found, but we don't know if it's optimal")
elif status == cp_model.INFEASIBLE:
    print("The problem was proven infeasible.")
elif status == cp_model.MODEL_INVALID:
    print("The given model didn't pass the validation step.")
elif status == cp_model.UNKNOWN:
    print("The status of the model is unknown because no solution was found")

An optimal solution was found.


In [4]:
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print("x =", solver.Value(x))
    print("y =", solver.Value(y))
    print("z =", solver.Value(z))
else:
    print("No solution found.")

x = 1
y = 0
z = 0


#### Finding all solutions

We modify the program above to find all feasible solutions.

In [5]:
class VarArraySolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, variables):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.__solution_count = 0

    def on_solution_callback(self):
        self.__solution_count += 1
        for v in self.__variables:
            print("{}={}".format(v, self.Value(v)), end=" ")
        print()

    def solution_count(self):
        return self.__solution_count

In [6]:
solver = cp_model.CpSolver()
solution_printer = VarArraySolutionPrinter([x, y, z])

# Enumerate all solutions.
solver.parameters.enumerate_all_solutions = True

# Solve.
status = solver.Solve(model, solution_printer)

x=1 y=0 z=0 
x=2 y=0 z=0 
x=2 y=0 z=1 
x=1 y=0 z=1 
x=2 y=1 z=1 
x=2 y=1 z=0 
x=2 y=1 z=2 
x=2 y=0 z=2 
x=1 y=0 z=2 
x=0 y=1 z=2 
x=0 y=1 z=1 
x=0 y=2 z=1 
x=0 y=2 z=2 
x=1 y=2 z=2 
x=1 y=2 z=1 
x=1 y=2 z=0 
x=0 y=2 z=0 
x=0 y=1 z=0 


In [7]:
print("Status = %s" % solver.StatusName(status))
print("Number of solutions found: %i" % solution_printer.solution_count())

Status = OPTIMAL
Number of solutions found: 18


### Solving a CP Problem

In [8]:
from ortools.sat.python import cp_model

# Declare the model.
model = cp_model.CpModel()

# Create the variables.
var_upper_bound = max(50, 45, 37)

x = model.NewIntVar(0, var_upper_bound, "x")
y = model.NewIntVar(0, var_upper_bound, "y")
z = model.NewIntVar(0, var_upper_bound, "z")

# Define the constraints.
model.Add(2 * x + 7 * y + 3 * z <= 50)
model.Add(3 * x - 5 * y + 7 * z <= 45)
model.Add(5 * x + 2 * y - 6 * z <= 37)

# Define the objective function.

model.Maximize(2 * x + 2 * y + 3 * z)

# Call the solver.
solver = cp_model.CpSolver()
status = solver.Solve(model)
status

4

In [9]:
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print(f"Maximum of objective function: {solver.ObjectiveValue()}\n")
    print(f"x = {solver.Value(x)}")
    print(f"y = {solver.Value(y)}")
    print(f"z = {solver.Value(z)}")
else:
    print("No solution found.")

Maximum of objective function: 35.0

x = 7
y = 3
z = 5


## Cryptarithmetic Puzzles

https://developers.google.com/optimization/cp/cryptarithmetic

In [10]:
from ortools.sat.python import cp_model

In [11]:
# Declare the model.
model = cp_model.CpModel()


# Defining the variables.
base = 10

c = model.NewIntVar(1, base - 1, "C")
p = model.NewIntVar(0, base - 1, "P")
i = model.NewIntVar(1, base - 1, "I")
s = model.NewIntVar(0, base - 1, "S")
f = model.NewIntVar(1, base - 1, "F")
u = model.NewIntVar(0, base - 1, "U")
n = model.NewIntVar(0, base - 1, "N")
t = model.NewIntVar(1, base - 1, "T")
r = model.NewIntVar(0, base - 1, "R")
e = model.NewIntVar(0, base - 1, "E")

# We need to group variables in a list to use the constraint AllDifferent.
letters = [c, p, i, s, f, u, n, t, r, e]

# Verify that we have enough digits.
assert base >= len(letters)

In [12]:
# Defining the constraints.
model.AddAllDifferent(letters)

# CP + IS + FUN = TRUE
model.Add(
    c * base + p + i * base + s + f * base * base + u * base + n
    == t * base * base * base + r * base * base + u * base + e
)

In [13]:
class VarArraySolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, variables):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.__solution_count = 0

    def on_solution_callback(self):
        self.__solution_count += 1
        for v in self.__variables:
            print("%s=%i" % (v, self.Value(v)), end=" ")
        print()

    def solution_count(self):
        return self.__solution_count

In [14]:
# Creates a solver and solves the model.
solver = cp_model.CpSolver()
solution_printer = VarArraySolutionPrinter(letters)

# Enumerate all solutions.
solver.parameters.enumerate_all_solutions = True

# Solve.
status = solver.Solve(model, solution_printer)

# Statistics.
print("\nStatistics")
print(f"  status   : {solver.StatusName(status)}")
print(f"  conflicts: {solver.NumConflicts()}")
print(f"  branches : {solver.NumBranches()}")
print(f"  wall time: {solver.WallTime()} s")
print(f"  sol found: {solution_printer.solution_count()}")

C=6 P=4 I=3 S=5 F=9 U=2 N=8 T=1 R=0 E=7 
C=3 P=4 I=6 S=5 F=9 U=2 N=8 T=1 R=0 E=7 
C=3 P=4 I=6 S=8 F=9 U=2 N=5 T=1 R=0 E=7 
C=3 P=2 I=6 S=7 F=9 U=8 N=5 T=1 R=0 E=4 
C=3 P=2 I=6 S=5 F=9 U=8 N=7 T=1 R=0 E=4 
C=2 P=3 I=7 S=6 F=9 U=8 N=5 T=1 R=0 E=4 
C=2 P=3 I=7 S=4 F=9 U=6 N=8 T=1 R=0 E=5 
C=2 P=3 I=7 S=5 F=9 U=4 N=8 T=1 R=0 E=6 
C=2 P=3 I=7 S=8 F=9 U=4 N=5 T=1 R=0 E=6 
C=2 P=3 I=7 S=8 F=9 U=6 N=4 T=1 R=0 E=5 
C=2 P=3 I=7 S=5 F=9 U=8 N=6 T=1 R=0 E=4 
C=7 P=3 I=2 S=5 F=9 U=8 N=6 T=1 R=0 E=4 
C=7 P=3 I=2 S=6 F=9 U=8 N=5 T=1 R=0 E=4 
C=6 P=2 I=3 S=7 F=9 U=8 N=5 T=1 R=0 E=4 
C=6 P=2 I=3 S=5 F=9 U=8 N=7 T=1 R=0 E=4 
C=7 P=3 I=2 S=4 F=9 U=6 N=8 T=1 R=0 E=5 
C=7 P=3 I=2 S=8 F=9 U=6 N=4 T=1 R=0 E=5 
C=7 P=4 I=2 S=8 F=9 U=6 N=3 T=1 R=0 E=5 
C=7 P=8 I=2 S=4 F=9 U=6 N=3 T=1 R=0 E=5 
C=7 P=8 I=2 S=3 F=9 U=6 N=4 T=1 R=0 E=5 
C=7 P=4 I=2 S=3 F=9 U=6 N=8 T=1 R=0 E=5 
C=7 P=6 I=2 S=3 F=9 U=8 N=5 T=1 R=0 E=4 
C=7 P=5 I=2 S=3 F=9 U=8 N=6 T=1 R=0 E=4 
C=7 P=8 I=2 S=3 F=9 U=4 N=5 T=1 R=0 E=6 
C=7 P=8 I=2 S=5 